In [ ]:
# documentation scipy.stats.norm: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

In [2]:
from scipy.stats import norm
import numpy as np

In [ ]:
# EMSR a with 3 classes
r = [300, 100, 50]  # prices
mu = [35, 25, 500]  # means
sigma = [2, 2, 2]  # sds
C = 160  # capacity

In [ ]:
# Values for G2
G_3_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[2]/r[0]),0)
G_3_2 = np.round(norm(mu[1], sigma[1]).ppf(1-r[2]/r[1]),0)
G2 = G_3_1 + G_3_2
B3 = C - G2

G1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0)  #ppf=percent point function (inverse of cumulative distribution function)
B2 = C-G1
B1 = C
G3 = C


In [ ]:
print("G1: {}".format(G1))
print("G2: {}".format(G2))
print("G3: {}".format(G3))
print("B1: {}".format(B1))
print("B2: {}".format(B2))
print("B3: {}".format(B3))


In [ ]:
# EMSR b with 3 classes
r = [300, 100, 50]  # prices
mu = [35, 25, 500]  # mean values
sigma = [2, 2, 2]  # SDs
C = 160  # capacity

In [ ]:
# G2
mu_2_est = mu[0] + mu[1] # estimated mu
sigma_2_est = np.sqrt(sigma[0]**2 + sigma[1]**2)  # estimated sigma
r_2_est = (mu[0]*r[0] + mu[1]*r[1])/(mu[0]+mu[1]) # weighted mean   
G2 = np.round(norm(mu_2_est, sigma_2_est).ppf(1-r[2]/r_2_est),0)
B3 = C-G2
G1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0)  #ppf=percent point function (inverse of cumulative distribution function)
B2 = C-G1
B1= C
G3 = C

In [ ]:
print("G1: {}".format(G1))
print("G2: {}".format(G2))
print("G3: {}".format(G3))
print("B1: {}".format(B1))
print("B2: {}".format(B2))
print("B3: {}".format(B3))

In [23]:
def emsr_a(c: int, r: list, mu: list, sigma: list):
    if len(r) != len(mu) or len(r) != len(sigma):
        raise ValueError('This does not work')
    
    num_classes = len(r)
    booking_limits = []
    protection_limits = []

    booking_limits.append(c) # Adding B_1 to the booking limits

    # From Class 1 to n-1
    for i in range(1, num_classes):
        G_limits = []
        for j in range(i):
            # Formula for G_i
            G_limits.append(np.round(norm(mu[j], sigma[j]).ppf(1-r[i]/r[j]),0))
        booking_limits.append(int(c-sum(G_limits)))  # Type conversion to int from numpy.int64
        protection_limits.append(int(sum(G_limits))) # Type conversion to int from numpy.int64

    protection_limits.append(c) # Adding G_n to the protection limits
    
    return booking_limits, protection_limits

# Test case
r = [300, 200, 100, 50, 25]  # ordered prices per class
mu = [10, 15, 25, 50, 500]  # ordered class demand means
sigma = [2, 2, 3, 3, 5]  # ordered class standard deviations
C = 160  # capacity
bl, pl = emsr_a(C, r, mu, sigma)
print(f'Booking limits: {bl}')
print(f'Protection limits: {pl}')

Booking limits: [160, 151, 134, 107, 53]
Protection limits: [9, 26, 53, 107, 160]


In [37]:
def emsr_b(c: int, r: list, mu: list, sigma: list):
    if len(r) != len(mu) or len(r) != len(sigma):
        raise ValueError('This does not work')
    
    num_classes = len(r)
    booking_limits = []
    protection_limits = []

    booking_limits.append(c) # Adding B_1 to the booking limits
    G1 = int(np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0))
    protection_limits.append(G1) # Adding G_1 to the protection limits
    booking_limits.append(c-G1) # Adding B_2 to the booking limits

    # From Class 1 to n-2 (Two classes are already done)
    for i in range(1, num_classes-1):
        mu_est = 0
        sigma_est = 0
        r_est = 0
        
        # Aggregation of the normal distributions
        for j in range(i+1):
            mu_est += mu[j]
            sigma_est += sigma[j]**2
            r_est += mu[j]*r[j]
        sigma_est = np.sqrt(sigma_est) # SQRT of the sum of the variances
        r_est = r_est/mu_est # Weighted mean
        G_i = int(np.round(norm(mu_est, sigma_est).ppf(1-r[i+1]/r_est),0))
        booking_limits.append(c-G_i) # Adding B_i+1 to the booking limits
        protection_limits.append(G_i) # Adding G_i to the protection limits

    protection_limits.append(c) # Adding G_n to the protection limits
    
    return booking_limits, protection_limits

r = [500, 400, 200, 100, 50, 25]  # ordered prices per class 
mu = [4, 8, 15, 25, 50, 500]  # ordered class demand means
sigma = [2, 2, 4, 10, 20, 50]  # ordered class standard deviations
C = 200  # capacity
bl, pl = emsr_b(C, r, mu, sigma)
print(f'Booking limits: {bl}')
print(f'Protection limits: {pl}')

Booking limits: [200, 198, 188, 171, 140, 78]
Protection limits: [2, 12, 29, 60, 122, 200]
